# Iterators and Generators

So far, you have seen things like

In [1]:
a = [1,2,3]
for x in a:
    print(x)

1
2
3


This looks very different from a C-style for loop where we loop over the variable index:
```C++
for (size_t i = 0; i < 3; i++) {
    printf("%d\n", i);
}
```

Or for instance, we can use something called a `range`

In [2]:
for i in range(3):
    print(i)

0
1
2


or other data types

In [3]:
d = {"hello": 1, "goodbye": 2}
for k,v in d.items():
    print(k, ' : ', v)

hello  :  1
goodbye  :  2


The key to using this sort of syntax is the concept of [iterator](https://wiki.python.org/moin/Iterator).  This is common in object-oriented programming (not just in Python), but you probably haven't seen iterators before if you've only used imperative languages.

An object is **iterable** if it implements the `__iter__` method, which is expected to return an iterator object.
An object is an **iterator** if it implements the `__next__` method, which either
1. returns the next element of the iterable object
2. raises the `StopIteration` exception if there are no more elements to iterate over

## A Basic Iterator

What if we want to replicate `range`?  

In [4]:
r = range(3)
type(r)

range

we can produce an iterator using the `iter` function

In [10]:
ri = iter(r)
type(ri)

range_iterator

we can explicitly run through the iterator using the `next` function

In [11]:
next(ri)

0

In [12]:
class my_range_iterator(object):
    def __init__(self, start, stop, stride):
        self.state = start
        self.stop = stop
        self.stride = stride
        
    def __next__(self):
        if self.state >= self.stop:
            raise StopIteration  # signals "the end"
        ret = self.state # we'll return current state
        self.state += self.stride # increment state
        return ret
        
        
# an iterable
class my_range(object):
    def __init__(self, start, stop, stride=1):
        self.start = start
        self.stop = stop
        self.stride = stride
    
    def __iter__(self):
        return my_range_iterator(self.start, self.stop, self.stride)

In [13]:
r = my_range(0,3)
type(r)

__main__.my_range

In [19]:
ri = iter(r)
type(ri)

__main__.my_range_iterator

In [20]:
next(ri)

0

In [21]:
for i in my_range(0,3):
    print(i)

0
1
2


In [22]:
for i in range(0,3):
    print(i)

0
1
2


You can also create classes that are both iterators and iterables

In [23]:
# an iterable and iterator
class my_range2(object):
    def __init__(self, start, stop, stride=1):
        self.start = start
        self.stop = stop
        self.stride = stride
        self.state = start
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.state >= self.stop:
            raise StopIteration  # signals "the end"
        ret = self.state # we'll return current state
        self.state += self.stride # increment state
        return ret

In [24]:
for i in my_range2(0,3):
    print(i)

0
1
2


## Using Iterators for Computation

Let's now use iterators for something more interesting - computing the Fibonacci numbers.

In [25]:
class FibonacciIterator(object):
    def __init__(self):
        self.a = 0 # current number
        self.b = 1 # next number
        
    def __iter__(self):
        return self
    
    def __next__(self):
        ret = self.a
        self.a, self.b = self.b, self.a + self.b # advance the iteration
        return ret

In [26]:
for i in FibonacciIterator():
    if i > 1000:
        break
    print(i)

0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987


Note that we never raise a `StopIteration` exception - the iterator will just keep going if we let it.

### Exercise

Define `FibonacciIterator` so it will iterate over all Fibonacci numbers until they are greater than a parameter `n`.

## Generators

Often, a more elegant way to define an iterator is using a [generator](https://wiki.python.org/moin/Generators)

This is a special kind of iterator defined using a function instead of using classes that implement the `__iter__` and `__next__` methods.

See [this post](https://nvie.com/posts/iterators-vs-generators/) for more discussion.

In [27]:
def my_range3(state, stop, stride=1):
    while state < stop:
        yield state
        state += stride
        

Note that we use the `def` keyword instead of the `class` keyword for our declaration.  The `yield` keyword returns subsequent values of the iteration.

In [28]:
r = my_range3(0,3)
type(r)

generator

In [29]:
ri = iter(r)
type(ri)

generator

In [33]:
next(ri)

StopIteration: 

In [34]:
for i in my_range3(0,3):
    print(i)

0
1
2


Our Fibonacci example re-written using a generator:

In [35]:
def FibonacciGenerator():
    a = 0
    b = 1
    while True:
        yield a
        a, b = b, a + b

In [36]:
for i in FibonacciGenerator():
    if i > 1000:
        break
    print(i)


0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987


### Exercise

Define `FibonacciGenerator` so it will iterate over all Fibonacci numbers until they are greater than a parameter `n`.

In [ ]:
## Your code here


## Iteration tools

Some useful tools for iterators that come in handy are:

`zip` - iterates over multiple iterators simulataneously

In [37]:
for i, a in zip([0,1,2], ['a', 'b', 'c']):
    print(i,a)

0 a
1 b
2 c


`reversed` - iterates in reverse order

In [38]:
for i in reversed(range(3)):
    print(i)

2
1
0


`enumerate` - returns the iteration step count as well as the iterator value

In [39]:
for i, a in enumerate('abc'):
    print(i, a)

0 a
1 b
2 c


### Exercise

Implement your own versions of `zip` and `enumerate` using generators

In [ ]:
## Your code here


## The Itertools Package

A useful package for dealing with iterators is the [itertools package](https://docs.python.org/3.8/library/itertools.html).  Here are a few examples - click on the link to see what else the package provides.

`product` gives the equivalent of a nested for-loop

In [40]:
from itertools import product

for i, j in product(range(2), range(3)):
    print(i, j)

0 0
0 1
0 2
1 0
1 1
1 2


`repeat` just repeats a value

In [41]:
from itertools import repeat

for i in repeat(10, 5):
    print(i)

10
10
10
10
10


`permutations`

In [42]:
from itertools import permutations

for p in permutations(range(3)):
    print(p)

(0, 1, 2)
(0, 2, 1)
(1, 0, 2)
(1, 2, 0)
(2, 0, 1)
(2, 1, 0)


### Exercise

Implement your own version of `product` and `repeat` using generators.

In [ ]:
## Your code here


## Iterators for Scientific Computing

One way you might use an iterator in scientific computing is when implementing an iterative algorithm.

Here is an example of the power method, which finds the largest eigenvalue-eigenvector pair of a matrix.

In [43]:
import numpy as np

def PowerMethodGenerator(A, x):
    
    def RayleighQuotient(A, x):
        return np.dot(x, A @ x) / np.dot(x,x)
    
    x = x / np.linalg.norm(x)
    rq_prev = np.inf
    rq = RayleighQuotient(A, x)
    
    while True:
        # yield state: RayleighQuotient, x, and difference from previous iteration
        yield rq, x, np.abs(rq - rq_prev)
        
        # compute next iteration
        x = A @ x
        x = x / np.linalg.norm(x)
        rq_prev = rq
        rq = RayleighQuotient(A, x)
    

In [44]:
n = 100
A = np.random.randn(n, n)
A = A @ A.T
x0 = np.random.randn(n)

solver = PowerMethodGenerator(A, x0)
tol = 1e-4

while True:
    rq, x, eps = next(solver)
    print(rq, eps)
    if eps < tol:
        break

112.64017380081526 inf
275.4063518492827 162.76617804846745
312.3214564784034 36.91510462912066
330.6061728280897 18.284716349686335
341.4077590815398 10.801586253450068
347.98627884619714 6.578519764657358
352.140827713928 4.154548867730853
354.90301925398626 2.762191540058268
356.8423690032573 1.9393497492710594
358.2721456462025 1.4297766429451713
359.3693558071367 1.0972101609341962
360.2385105611948 0.8691547540581155
360.9444828792367 0.7059723180419155
361.5295385330741 0.5850556538374008
362.0224165809222 0.4928780478480803
362.4433748623568 0.4209582814345936
362.8071274640744 0.36375260171763557
363.12463504792623 0.3175075838518069
363.4042414236916 0.2796063757653542
363.6524201333118 0.2481787096202197
363.874278707542 0.22185857423016841
364.07390716000674 0.1996284524647649
364.25462361965407 0.18071645964732852
364.41915056483464 0.16452694518056887
364.5697434089549 0.15059284412024
364.70828587268 0.13854246372511625
364.836361879778 0.12807600709800226
364.9553106320

If we decide that we're not satisfied with convergence yet, we can resume where we left off

In [45]:
tol = 1e-6

while True:
    rq, x, eps = next(solver)
    print(rq, eps)
    if eps < tol:
        break

367.9383283674861 9.31665379084734e-05
367.93841796867406 8.960118793766014e-05
367.938504140868 8.617219396001019e-05
367.9385870152154 8.287434735620991e-05
367.93866671785395 7.970263857259852e-05
367.9387433701017 7.665224774200396e-05
367.9388170886417 7.37185400225826e-05
367.9388879856983 7.089705655971557e-05
367.938956169207 6.818350874482348e-05
367.93902174297847 6.557377145099963e-05
367.93908480685536 6.306387689392068e-05
367.93914545686425 6.065000889066141e-05
367.93920378536063 5.832849637954496e-05
367.9392598811699 5.609580927057323e-05
367.9393138297209 5.394855099893903e-05
367.9393657131767 5.1883455796541966e-05
367.9394156105586 4.989738192762161e-05
367.93946359786617 4.7987307539187896e-05
367.93950974819285 4.6150326681981824e-05
367.9395541318368 4.4383643967194075e-05
367.93959681640786 4.268457104217305e-05
367.9396378669296 4.105052175873425e-05
367.939677345939 3.9479009387832775e-05
367.9397153135808 3.796764178787271e-05
367.93975182769924 3.6514118448